# Mnist in libtorch

In [1]:
#pragma cling add_library_path("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/lib/")
#pragma cling add_include_path("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/include")
#pragma cling add_include_path("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/include/torch/csrc/api/include/")

In [2]:
#pragma cling load("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/lib/libc10.so")
#pragma cling load("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/lib/libgomp-75eea7e8.so.1")
#pragma cling load("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/lib/libtorch.so")

In [3]:
#include <torch/torch.h>
#include <iostream>
#include <typeinfo>

## Flatten()

Utility function to flatten `torch::Tensor` for FC layers

In [4]:
torch::Tensor Flatten(torch::Tensor x){
    return x.view({x.sizes()[0], -1});
}

## Network

Defining the network, In Python it was a class which inherited the `torch.nn.Module`. In C++ it's the same, inplace of a class here we define a `struct` which inherits `torch::nn::Module`

Similarities and differences between Python API and C++:
- In the constructor for the `struct` we define all the layers just like we do in Python API.
- The `forward` method is the same as in Python 

In [5]:
struct Net: torch::nn::Module {
    Net()
        : conv1(torch::nn::Conv2dOptions(1, 6, 3)), 
    conv2(torch::nn::Conv2dOptions(6, 16, 3)), 
    fc1(576, 120), 
    fc2(120, 84), fc3(84, 10){
        register_module("conv1", conv1);
        register_module("conv2", conv2);
        register_module("fc1", fc1);
        register_module("fc2", fc2);
        register_module("fc3", fc3);
    }
    
    torch::Tensor forward(torch::Tensor x){
        x = torch::max_pool2d(torch::relu(conv1->forward(x)), 2);
        x = torch::max_pool2d(torch::relu(conv2->forward(x)), 2);
        x = Flatten(x);
        x = torch::relu(fc1->forward(x));
        x = torch::relu(fc2->forward(x));
        x = fc3->forward(x);
        return x;
    }
    
    torch::nn::Conv2d conv1;
    torch::nn::Conv2d conv2;
    torch::nn::Linear fc1;
    torch::nn::Linear fc2;
    torch::nn::Linear fc3;
}

In [6]:
Net model;

In [7]:
torch::Tensor ip = torch::randn({1, 1, 32, 32});
auto output = model.forward(ip);

In [8]:
std::cout << output;

 0.1640 -0.0801  0.0833  0.0340  0.1296 -0.0128  0.0118  0.0391 -0.1252  0.1189
[ CPUFloatType{1,10} ]

In [9]:
template<typename DataLoader>
void trainer(DataLoader& loader, Net& net){
    for(auto &batch: loader){
        auto images = batch.data, targets = batch.target;
        std::cout << images.sizes() << " " << targets.sizes(); 
    }
}

In [10]:
void test(){
    
}

In [11]:
auto train_dataset = torch::data::datasets::MNIST("./data").map(torch::data::transforms::Stack<>());

In [21]:
std::cout << train_dataset.size().value();

60000

In [12]:
const size_t train_dataset_size = train_dataset.size().value();

In [13]:
std::cout << train_dataset_size; 

60000

In [14]:
auto item = train_dataset.get(0)

input_line_20:2:28: error: no member named 'get' in 'torch::data::datasets::MapDataset<torch::data::datasets::MNIST,
      torch::data::transforms::Stack<Example<> > >'
 auto item = train_dataset.get(0)
             ~~~~~~~~~~~~~ ^


Interpreter Error: 

In [ ]:
std::cout << item.data.sizes();

In [ ]:
auto train_loader = torch::data::make_data_loader<torch::data::samplers::RandomSampler>(
    std::move(train_dataset), 4);

In [ ]:
auto& batch = *train_loader;

In [ ]:
typeid(batch).name()